In [1]:
import numpy as np

In [64]:
class FOS:
    def __init__(self, n_class: int):
        self.k = n_class
    
    def mean(self, X: np.ndarray, low: int, high: int):
        return X[low-1: high].mean(axis=0)
    
    def varition(self, X: np.ndarray, low: int, high: int):
        data = X[low-1: high]
        devia = data - self.mean(X, low, high)
        vari = (devia ** 2).sum()
        return vari
    
    def fit_transform(self, X: np.ndarray):
        table = self.split_k(X)
        split_points = table[self.k, X.shape[0]][1]
        tmp = split_points
        split_points = np.insert(split_points, 0, 1)
        split_points = np.append(split_points, X.shape[0]+1)
        split_res = [[split_points[i], split_points[i+1] - 1] for i in range(len(split_points)-1)]
        return split_res, tmp, table
    
    def split_k(self, X: np.ndarray):
        n = X.shape[0]
        table = np.zeros((self.k+1, n+1), dtype='object')
        for j in range(2, n+1):
            table[2, j] = self.split_2(X, j)
        for i in range(3, self.k+1):
            for j in range(i, n+1):
                cache = {}
                for m in range(i-1, j):
                    loss = table[i-1, m][0] + self.varition(X, m+1, j)
                    cache[loss] = m
                min_loss = min(cache.keys())
                min_cutp = cache[min_loss]
                table[i, j] = (min_loss, np.append(table[i-1, min_cutp][1], min_cutp+1))
        return table
    
    def split_2(self, X: np.ndarray, endp: int):
        loss2cutp = {}
        for i in range(2, endp+1):
            loss2cutp[self.varition(X, 1, i-1) + self.varition(X, i, endp)] = i
        min_loss = min(loss2cutp.keys())
        min_cutp = loss2cutp[min_loss]
        res = (min_loss, np.array([min_cutp]))
        return res
    

In [76]:
data = np.random.randint(10, size=(1000, 2))

In [77]:
fos = FOS(4)
res, ps, table = fos.fit_transform(data)
res, ps

([[1, 32], [33, 44], [45, 918], [919, 1000]], array([ 33,  45, 919]))